# Convert SQLite output to parquet file with CytoTable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# Add the PathName columns separately
joins = joins.replace(
    "COLUMNS('Image_FileName_.*'),",
    "COLUMNS('Image_FileName_.*'),\n COLUMNS('Image_PathName_.*'),",
)

# type of file output
dest_datatype = "parquet"

# Set directory if processing redo plate
redo_plate = True
if redo_plate:
    # set path to directory with SQLite files
    sqlite_dir = pathlib.Path("../2.extract_features/cp_output/Plate_2_redo")
    # directory for processed data
    output_dir = pathlib.Path("data/converted_profiles/Plate_2_redo")
    output_dir.mkdir(parents=True, exist_ok=True)
else:
    # set path to directory with SQLite files
    sqlite_dir = pathlib.Path("../2.extract_features/cp_output")
    # directory for processed data
    output_dir = pathlib.Path("data/converted_profiles")
    output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

# Select plate name folders if starts with CARD
for file_path in sqlite_dir.iterdir():
    if file_path.is_dir() and file_path.name.startswith("CARD"):
        plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 1 plates in this dataset. Below are the names:
CARD-CelIns-CX7_251110170001


## Convert SQLite to parquet files

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(f"{output_dir}/{file_path.stem}_converted.parquet")
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # Merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000,
    )

print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: CARD-CelIns-CX7_251110170001
All plates have been converted with cytotable!


# Load in converted profiles to update

In [4]:
for file_path in output_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # If any, drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])

    # Rearrange columns and add "Metadata" prefix in one line
    df = df[
        [
            "Nuclei_Location_Center_X",
            "Nuclei_Location_Center_Y",
            "Cells_Location_Center_X",
            "Cells_Location_Center_Y",
            "Image_Count_Cells",
        ]
        + [
            col
            for col in df.columns
            if col
            not in [
                "Nuclei_Location_Center_X",
                "Nuclei_Location_Center_Y",
                "Cells_Location_Center_X",
                "Cells_Location_Center_Y",
                "Image_Count_Cells",
            ]
        ]
    ].rename(
        columns=lambda col: (
            "Metadata_" + col
            if col
            in [
                "Nuclei_Location_Center_X",
                "Nuclei_Location_Center_Y",
                "Cells_Location_Center_X",
                "Cells_Location_Center_Y",
                "Image_Count_Cells",
            ]
            else col
        )
    )

    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)

## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [5]:
print(df.shape)
df.head()

(9933, 2476)


,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,658.950181,133.526354,655.840801,145.906197,8,8,CARD-CelIns-CX7_251110170001,f11,B02,1,...,7.235763,6.820124,28.790843,25.668908,27.056171,28.936658,2.489488,2.524561,2.714171,2.487867
1,447.021930,78.739348,484.574235,96.405302,8,11,CARD-CelIns-CX7_251110170001,f17,B02,1,...,14.929112,14.343909,92.442490,98.489600,89.491760,82.556143,12.022663,12.248467,11.881171,11.092255
2,901.166307,143.648668,899.842264,187.085445,12,39,CARD-CelIns-CX7_251110170001,f00,B04,1,...,12.097805,12.197471,1.964058,1.945975,2.083970,2.101056,7.077019,6.946078,6.867486,6.768404
3,298.035906,54.772776,288.291241,114.819708,17,91,CARD-CelIns-CX7_251110170001,f07,B07,1,...,3.313076,3.460958,0.857484,0.847633,0.934912,0.961085,3.656769,3.795006,3.590538,3.668976
4,621.179219,88.804742,572.228177,149.077273,15,100,CARD-CelIns-CX7_251110170001,f18,B07,1,...,10.112048,10.461192,1.288340,1.416899,1.288473,1.196812,12.661273,10.548908,12.318059,13.956980
